# Queries with and without Azure OpenAI

So far, you have your Search Engine loaded **from two different data sources in two diferent indexes**, on this notebook we are going to try some example queries and then use Azure OpenAI service to see if we can get a good answer for the user query.

The idea is that a user can ask a question about Computer Science (first datasource/index) or about Covid (second datasource/index), and the engine will respond accordingly.
This **Multi-Index** demo, mimics the scenario where a company loads multiple type of documents of different types and about completly different topics and the search engine must respond with the most relevant results.

## Set up variables

In [1]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from typing import List
from operator import itemgetter

# LangChain Imports needed
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.runnables import ConfigurableField


# Our own libraries needed
from common.prompts import DOCSEARCH_PROMPT
from common.utils import get_search_results

from dotenv import load_dotenv
load_dotenv("credentials.env")

True

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Multi-Index Search queries

In [4]:
# Text-based Indexes that we are going to query (from Notebook 01 and 02)
index1_name = "srch-index-books"
index2_name = "srch-index-usecases"
indexes = os.environ['COG_INDEXES']
print(indexes)

["srch-index-usecases", "srch-index-csv","srch-index-books"]


Try questions that you think might be answered or addressed in computer science papers in 2020-2021 or that can be addressed by medical publications about COVID in 2020-2021. Try comparing the results with the open version of ChatGPT.<br>
The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

**Example Questions you can ask**:
- What is CLP?
- How Markov chains work?
- What are some examples of reinforcement learning?
- What are the main risk factors for Covid-19?
- What medicine reduces inflamation in the lungs?
- Why Covid doesn't affect kids that much compared to adults?
- Does chloroquine really works against covid?
- Who won the 1994 soccer world cup? # This question should yield no answer if the system is correctly grounded

In [5]:
QUESTION = " What does PL-BERT do excatly?"

### Search on both indexes individually and aggragate results

#### **Note**: 
In order to standarize the indexes, **there must be 6 mandatory fields present on each index**: `id, title, name, location, chunk, chunkVector`. This is so that each document can be treated the same along the code. Also, **all indexes must have a semantic configuration**.

We are going to use Hybrid Queries: Text + Vector Search combined for optimal results!

In [6]:
agg_search_results = dict()
k = 10

for index in indexes:
    search_payload = {
        "search": QUESTION, # Text query
        "select": "id, title, name, location, chunk",
        "queryType": "semantic",
        "vectorQueries": [{"text": QUESTION, "fields": "chunkVector", "kind": "text", "k": k, 
                           "threshold": { 
                                 "kind": "vectorSimilarity", 
                                 "value": 0.8 
                              }
                          }], # Vector query
        "semanticConfiguration": "my-semantic-config",
        "captions": "extractive",
        "answers": "extractive",
        "count":"true",
        "top": k
    }

    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

400


KeyError: '@odata.count'

#### Important Note: 
You may encounter errors when attempting to search for results IF the indexer is still processing documents. This occurs because the embedding model is heavily utilized by the indexer, hitting its TPM quota. If you experience search errors, please try again or wait until the indexing is complete, which may take several hours.

In [6]:
agg_search_results

{'srch-index-csv': {'@odata.context': "https://cog-search-ffpanhhmq7wy3.search.windows.net/indexes('srch-index-csv')/$metadata#docs(*)",
  '@odata.count': 1011,
  '@search.answers': [],
  'value': [{'@search.score': 0.03333333507180214,
    '@search.rerankerScore': 3.151348114013672,
    '@search.captions': [{'text': "DB-20181219-mener-maleriet-gir-h-5. Noen preges mer av kunsten enn andre. En 70 år gammel mann ble lørdag rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis for mannen var en gruppe kardiologer også på besøk i galleriet, og behandlet ham på stedet. Dette melder The Times. Etter sigende er det verken rødt kjøtt eller sigaretter som forårsaket anfallet. Tvert imot kan vi ifølge avisa skylde på renessansemaleren Sandro Botticelli, og maleriet hans «Venus' fødsel» fra 1486..",
      'highlights': "DB-20181219-mener-maleriet-gir-h-5.<em> Noen preges mer av kunsten enn andre

### Display the top results (from both searches) based on the score

In [7]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():

    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))

            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunk": result['chunk'], 
                                    "name": result['name'], 
                                    "location": result['location'] ,
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "index": index
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = str(ordered_content[id]['location']) + os.environ['BLOB_SAS_TOKEN']
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

### Comments on Query results

As seen above the semantic re-ranking feature of Azure AI Search service is good. It gives answers (sometimes) and also the top results with the corresponding file and the paragraph where the answers is possible located.

Let's see if we can make this better with Azure OpenAI

# Using Azure OpenAI

To use OpenAI to get a better answer to our question, the thought process is simple: let's **give the answer and the content of the documents from the search result to the GPT model as context and let it provide a better response**. This is what RAG (Retreival Augmented Generation) is about.

Now, before we do this, we need to understand a few things first:

1) Chainning and Prompt Engineering
2) Embeddings

We will use a library call **LangChain** that wraps a lot of boiler plate code.
Langchain is one library that does a lot of the prompt engineering for us under the hood, for more information see [here](https://python.langchain.com/en/latest/index.html)

In [8]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

**Important Note**: Starting now, we will utilize OpenAI models. Please ensure that you have deployed the following models within the Azure OpenAI portal:

- text-embedding-ada-002 (or newer)
- gpt-35-turbo (1106 or newer)
- gpt-4-turbo (1106 or newer)

Reference for Azure OpenAI models (regions, limits, dimensions, etc): [HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

## A gentle intro to chaining LLMs and prompt engineering

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

Azure OpenAI is a type of LLM (provider) that you can use but there are others like Cohere, Huggingface, etc.

Chains can be simple (i.e. Generic) or specialized (i.e. Utility).

* Generic — A single LLM is the simplest chain. It takes an input prompt and the name of the LLM and then uses the LLM for text generation (i.e. output for the prompt).

Here’s an example:

In [9]:
COMPLETION_TOKENS = 2000
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                      temperature=0, 
                      max_tokens=COMPLETION_TOKENS)

llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe07a4a2530>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe07a515120>, root_client=<openai.lib.azure.AzureOpenAI object at 0x7fe094185fc0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7fe07a4a2590>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=2000, azure_endpoint='https://11092024test.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-05-01-preview', openai_api_type='azure')

In [10]:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}. Give your response in {language}")
])

The | symbol is similar to a unix pipe operator, which chains together the different components feeds the output from one component as input into the next component.

In [11]:
chain = prompt | llm | output_parser

In [12]:
%%time
display(Markdown(chain.invoke({"input": QUESTION, "language": "Spanish"})))

Un hombre de 70 años experimentó un malestar repentino mientras visitaba la Galería Uffizi en Florencia, Italia. Este tipo de reacción se conoce a veces como el "Síndrome de Stendhal", una condición que puede ocurrir cuando alguien se siente abrumado por la belleza del arte. Los síntomas pueden incluir mareos, palpitaciones y desmayos. En este caso, el hombre fue atendido rápidamente por el personal del museo y los servicios médicos. Afortunadamente, se recuperó sin complicaciones graves.

CPU times: user 17.2 ms, sys: 7.48 ms, total: 24.6 ms
Wall time: 1.28 s


**Note**: this is the first time you use OpenAI in this Accelerator, so if you get a Resource not found error, is most likely because the name of your OpenAI model deployment is different than the environmental variable set above `os.environ["GPT35_DEPLOYMENT_NAME"]`

Great!!, now you know how to create a simple prompt and use a chain in order to answer a general question using ChatGPT knowledge!. 

It is important to note that we rarely use generic chains as standalone chains. More often they are used as building blocks for Utility chains (as we will see next). Also important to notice is that we are NOT using our documents or the result of the Azure Search yet, just the knowledge of ChatGPT on the data it was trained on.

**The second type of Chains are Utility:**

* Utility — These are specialized chains, comprised of many building blocks to help solve a specific task. For example, LangChain supports some end-to-end chains (such as `create_retrieval_chain` for QnA Doc retrieval, Summarization, etc).

We will build our own specific chain in this workshop for digging deeper and solve our use case of enhancing the results of Azure AI Search.


But before dealing with the utility chain needed, let's first review the concept of Embeddings and Vector Search and RAG. 

## Embeddings and Vector Search

From the Azure OpenAI documentation ([HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/embeddings?tabs=python)), An embedding is a special format of data representation that can be easily utilized by machine learning models and algorithms. The embedding is an information dense representation of the semantic meaning of a piece of text. Each embedding is a vector of floating point numbers, such that the distance between two embeddings in the vector space is correlated with semantic similarity between two inputs in the original format. For example, if two texts are similar, then their vector representations should also be similar. 

### Why Do We Need Vectors?

Vectors are essential for several reasons:

- **Semantic Richness**: They convert the semantic meaning of text into mathematical vectors, capturing nuances that simple keyword searches miss. This makes them incredibly powerful for understanding and processing language.
- **Human-like Searching**: Searching using vector distances mimics the human approach to finding information based on context and meaning, rather than relying solely on exact word matches.
- **Efficiency in Scale**: Vector representations allow for efficient handling and searching of large datasets. By reducing complex text to numerical vectors, algorithms can quickly sift through vast amounts of information.

### Understanding LLM Tokens' Context Limitation

Large Language Models (LLMs) like GPT come with a token limit for each input, which poses a challenge when dealing with lengthy documents or extensive data sets. This limitation restricts the model's ability to understand and generate responses based on the full context of the information provided. It becomes crucial, therefore, to devise strategies that can effectively manage and circumvent this limitation to leverage the full power of LLMs.

To address this challenge, the solution incorporates several key steps:

1. **Segmenting Documents**: Breaking down large documents into smaller, manageable segments.
2. **Vectorization of Chunks**: Converting these segments into vectors, making them compatible with vector-based search techniques.
3. **Hybrid Search**: Employing both vector and text search methods to pinpoint the most relevant segments in relation to the query.
4. **Optimal Context Provision**: Presenting the LLM with the most pertinent segments, ensuring a balance between detail and brevity to stay within token limits.


Our ultimate goal is to rely solely on vector indexes and hybrid searchs (vector + text). While it is possible to manually code parsers with OCR for various file types and develop a scheduler to synchronize data with the index, there is a more efficient alternative: **Azure AI Search has automated chunking strategies and vectorization**.

It's important to note that **document segmentation and vectorization have already been completed in AI Azure Search**, as seen in the `ordered_content` dictionary. This pre-processing step simplifies subsequent operations, ensuring rapid response times and adherence to the token limits of the chosen OpenAI model.


So really, our only job now is to make sure that the results from the Azure AI Search queries fit on the LLM context size, and then let it do its magic.

In [15]:
index_name = "srch-index-files"
index2_name = "srch-index-csv"
indexes = [index2_name]

In order to not duplicate code, we have put many of the code used above into functions. These functions are in the `common/utils.py` and `common/prompts.py` files. This way we can use these functios in the app that we will build later.

`get_search_results()` do the multi-index search and returns the combined ordered list of documents/chunks.

In [16]:
k = 20  # play with this parameter and see the quality of the final answer
ordered_results = get_search_results(QUESTION, indexes, k=k, reranker_threshold=1)
print("Number of results:",len(ordered_results))

Number of results: 19


In [17]:
# Uncomment the below line if you want to inspect the ordered results
ordered_results

OrderedDict([('DB-20181219-mener-maleriet-gir-h-5',
              {'title': 'DB-20181219-mener-maleriet-gir-h-5',
               'name': 'newspapers_online_nb',
               'chunk': "Noen preges mer av kunsten enn andre. En 70 år gammel mann ble lørdag rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis for mannen var en gruppe kardiologer også på besøk i galleriet, og behandlet ham på stedet. Dette melder The Times. Etter sigende er det verken rødt kjøtt eller sigaretter som forårsaket anfallet. Tvert imot kan vi ifølge avisa skylde på renessansemaleren Sandro Botticelli, og maleriet hans «Venus' fødsel» fra 1486.",
               'location': '2021',
               'caption': "DB-20181219-mener-maleriet-gir-h-5. Noen preges mer av kunsten enn andre. En 70 år gammel mann ble lørdag rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide 

Now let's create a Prompt Template that will ground the response only in the chunks retrieve by our hybrid AI Search.

In [18]:
template = """Answer the question thoroughly, based **ONLY** on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [21]:
%%time 
# Creation of our custom chain
chain = prompt | llm | output_parser

print(QUESTION)
try:
    display(Markdown(chain.invoke({"question": QUESTION, "context": ordered_results})))
except Exception as e:
    print(e)

Noen preges mer av kunsten enn andre. En 70 år gammel mann ble lørdag rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Hva skjedde? 


En 70 år gammel mann ble rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det var en hjertestans eller et hjerteinfarkt. Heldigvis var en gruppe kardiologer til stede og behandlet ham på stedet. Ifølge rapporter kan anfallet ha vært forårsaket av renessansemaleren Sandro Botticellis maleri «Venus' fødsel» fra 1486, og ikke av rødt kjøtt eller sigaretter.

CPU times: user 17.7 ms, sys: 801 μs, total: 18.5 ms
Wall time: 1.76 s


### From GPT-3.5 to GPT-4

Now let's see how the response changes if we change to GPT-4


In [18]:
llm_2 = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS)
chain = prompt | llm_2 | output_parser

In [19]:
%%time
try:
    display(Markdown(chain.invoke({"question": QUESTION, "context": ordered_results})))
except Exception as e:
    print(e)

Based on the provided context, several medicines and treatments have been identified to reduce inflammation in the lungs:

1. **Interleukin-22 (IL-22):** Administering recombinant IL-22 in vivo reduces inflammation and fluid leak into the lung, demonstrating that the IL-22/IL-22BP axis is a potential pathway for reducing influenza-induced pneumonia.

2. **Chloroquine:** In human lung parenchymal explants, a clinically achievable concentration of chloroquine inhibited the lipopolysaccharide-induced release of inflammatory cytokines (TNF-α, IL-6, CCL2, and CCL3) significantly, suggesting its role in mitigating the cytokine storm associated with severe pneumonia caused by coronaviruses.

3. **Escin:** Suggested for its potent anti-inflammatory and anti-viral effects in lung injury, indicating it could be a therapeutic opportunity as add-on therapy in Acute Lung Injury (ALI) related to COVID-19 infection.

4. **Corticosteroids:** Known for their anti-inflammatory effects, corticosteroids are considered effective in chronic inflammatory and immune diseases like asthma, though their effectiveness varies among different patient groups.

5. **Tanshinone IIA (TIIA):** Demonstrated to notably attenuate Bleomycin-induced Pulmonary Fibrosis (PF) and inflammation in rats, indicating it may exert protective effects through the ACE-2/ANG-(1-7) axis.

6. **Low doses of radiation therapy (RT):** Shown to increase the immunosuppressive profile of lung macrophages via IL-10 production and IFNγ/IL-6 suppression, suggesting a therapeutic strategy to counteract lung inflammation.

7. **Tocilizumab (Interleukin-6 blockade):** Demonstrated to curb the "cytokine storm," prevent ICU admission, and the requirement for mechanical ventilation in highly selected patients with severe COVID-19.

8. **Antibiotics (Macrolides and Ketolides):** Indicated for their anti-inflammatory effects on the chronically inflamed airways in addition to their anti-infective action, especially in the context of asthma exacerbated by atypical respiratory pathogens.

9. **Noscapine:** Used for the treatment of cough, has been shown to inhibit bradykinin enhanced cough response, indicating a potential role in attenuating cytokine release associated with SARS-CoV-2.

These treatments vary in their mechanism of action, from directly targeting cytokines involved in inflammation (like IL-22 and IL-6 blockade) to more broad anti-inflammatory effects (such as corticosteroids and chloroquine). Some are specific to certain conditions (e.g., Escin for COVID-19 related ALI, TIIA for pulmonary fibrosis), while others have a more general application in reducing lung inflammation.

CPU times: user 19.3 ms, sys: 533 µs, total: 19.9 ms
Wall time: 15.9 s


#### As we can see, the model selection MATTERS!

We will dive deeper into this later, but for now, **look at the diference between GPT3.5 and GPT4, in quality and in response time**.

# Improving the Prompt and adding citations

We could see above that the answer given by GPT3.5 was very simple compared to GPT4, even when the prompt says "thorough responses to users". We also could see that there is no citations or references. **How do we know if the answer is grounded on the context or not?**

Let's see if these two issues can be improved by Prompt Engineering.<br>
On `common/prompts.py` we created a prompt called `DOCSEARCH_PROMPT` check it out!

**Let's also create a custom Retriever class** so we can plug it in easily within the chain building. 
Note: we can also use the Azure AI Search retriever class [HERE](https://python.langchain.com/docs/integrations/vectorstores/azuresearch), however we want to create a custom Retriever for the following reasons:

1) We want to do multi-index searches in one call
2) Easier to teach complex concepts of LangChain in this notebook
3) We want to use the REST API vs the Python Azure Search SDK

In [22]:
class CustomRetriever(BaseRetriever):
    
    topK : int
    reranker_threshold : int
    indexes: List
    sas_token: str = None
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        
        ordered_results = get_search_results(query, self.indexes, k=self.topK, 
                                             reranker_threshold=self.reranker_threshold, 
                                             sas_token=self.sas_token)
        top_docs = []
        for key,value in ordered_results.items():
            location = value["location"] if value["location"] is not None else ""
            top_docs.append(Document(page_content=value["chunk"], metadata={"source": location, "score":value["score"]}))

        return top_docs

In [23]:
# Create the retriever
retriever = CustomRetriever(indexes=indexes, topK=k, reranker_threshold=1, sas_token=os.environ['BLOB_SAS_TOKEN'])

In [24]:
# Test retreiver
results = retriever.invoke(QUESTION)
len(results)

19

In [25]:
# We can create now a dynamically configurable llm object that can change the model at runtime
dynamic_llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                              temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="model"),
    # This sets a default_key.
    # If we specify this key, the default LLM  (initialized above) will be used
    default_key="gpt35",
    # This adds a new option, with name `gpt4`
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                         temperature=0.5, max_tokens=COMPLETION_TOKENS),
    gpt4o=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                         temperature=0.5, max_tokens=COMPLETION_TOKENS),
    # You can add more configuration options here
)

In [26]:
# Declaration of the chain with the dynamic llm and the new prompt
configurable_chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the input variables above to the prompt template
    | dynamic_llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

In [27]:
%%time

try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt35"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

En 70 år gammel mann fikk et akutt illebefinnende mens han besøkte Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis var en gruppe kardiologer også til stede i galleriet og behandlet ham på stedet. Hendelsen ble ikke forårsaket av rødt kjøtt eller sigaretter, men ble tilskrevet renessansemaleren Sandro Botticelli og hans maleri «Venus' fødsel» fra 1486 [[1]](https://example.com/2021sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-09-19T15:56:50Z&st=2024-09-12T07:56:50Z&spr=https,http&sig=58L%2FTEjMFajQDeoAbH7R5UkwuTxllvEoHpsANoFhmeU%3D).

CPU times: user 26.6 ms, sys: 4.45 ms, total: 31.1 ms
Wall time: 3.94 s


As seen above, we were able to improve the quality and breath of the answer and add citations with only prompt engineering!

#### Let's try again, but with GPT-4

In [28]:
%%time
try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt4"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

En 70 år gammel mann ble rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis var en gruppe kardiologer også til stede i galleriet og behandlet ham på stedet. Hendelsen ble rapportert av The Times, og det ble antydet at anfallet kunne ha vært forårsaket av renessansemaleren Sandro Botticelli og hans maleri «Venus' fødsel» fra 1486, snarere enn av rødt kjøtt eller sigaretter [[1]](https://example.com/2021sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-09-19T15:56:50Z&st=2024-09-12T07:56:50Z&spr=https,http&sig=58L%2FTEjMFajQDeoAbH7R5UkwuTxllvEoHpsANoFhmeU%3D).

CPU times: user 27.9 ms, sys: 1.37 ms, total: 29.2 ms
Wall time: 4.11 s


#### And GPT-4o

In [29]:
%%time
try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt4o"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

En 70 år gammel mann ble rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis var en gruppe kardiologer også på besøk i galleriet og behandlet ham på stedet. Ifølge rapporter fra The Times, var det ikke rødt kjøtt eller sigaretter som forårsaket anfallet, men renessansemaleren Sandro Botticelli og hans maleri «Venus' fødsel» fra 1486 som ble nevnt som en mulig årsak [[1]](https://example.com/2021sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-09-19T15:56:50Z&st=2024-09-12T07:56:50Z&spr=https,http&sig=58L%2FTEjMFajQDeoAbH7R5UkwuTxllvEoHpsANoFhmeU%3D).

CPU times: user 29 ms, sys: 0 ns, total: 29 ms
Wall time: 4.1 s


#### As you can see the answer from GPT4 is richer, and includes all the relevant chunks. GPT3.5 tends to focus in the first and last chunks only

## Adding Streaming to improve user experience and performance

It is obvious by now that **GPT4 answers are better quality than GPT3.5**. None are incorrect, but GPT4 is better at understanding the context, following the prompt instructions and on giving a comprehensive answer.

One way to make GPT4 look faster is to stream the answer, so the user can see the response as it is typed. To do this, we just simply need to call the method `stream` instead of `invoke`. More on Streaming and Callbacks in later notebooks, but for now, this is one simple way to do it:

In [30]:
for chunk in configurable_chain.with_config(configurable={"model": "gpt4o"}).stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

En 70 år gammel mann ble rammet av et akutt illebefinnende i Uffizi-galleriet i Firenze, Italia. Det er uklart om det dreide seg om en hjertestans eller et hjerteinfarkt. Heldigvis var en gruppe kardiologer også på besøk i galleriet og behandlet ham på stedet. Ifølge The Times, var anfallet ikke forårsaket av rødt kjøtt eller sigaretter, men heller av renessansemaleren Sandro Botticellis maleri «Venus' fødsel» fra 1486 [[1]](https://example.com?query_parameters).

# Using Other models from the Azure AI Catalog (optional)

Besides OpenAI models, Azure AI Studio provides a wide selection of models from an extensive catalog featuring providers such as Mistral, Cohere, Meta, HuggingFace, Nvidia, Databricks, and Microsoft's own offerings.

Several models are available as serverless, pay-as-you-go endpoints. Examples include Mistral-Large and Cohere Command R+.

To use the following code, visit https://ai.azure.com and deploy [Mistral-Large](https://ai.azure.com/explore/models/Mistral-large/version/1/registry/azureml-mistral) and [Cohere-Command-R-plus](https://ai.azure.com/explore/models/Cohere-command-r-plus/version/3/registry/azureml-cohere) on a pay-as-you-go basis, then copy the endpoint URLs and keys into the cells below.

In [29]:
from langchain_mistralai import ChatMistralAI
from langchain_cohere.chat_models import ChatCohere

In [30]:
os.environ["MISTRAL_API_ENDPOINT"] = "https://<YOUR_MISTRAL_DEPLOYMENT_NAME>.<YOUR_REGION>.inference.ai.azure.com"
os.environ["MISTRAL_API_KEY"] = "ENTER HERE YOUR MISTRAL ENDPOINT API KEY"
os.environ["COHERE_API_ENDPOINT"] = "https://<YOUR_COHERE_DEPLOYMENT_NAME>.<YOUR_REGION>.inference.ai.azure.com/v1"
os.environ["COHERE_API_KEY"] = "ENTER HERE YOUR COHERE ENDPOINT API KEY"

In [31]:
llm_mistral = ChatMistralAI(endpoint=os.environ["MISTRAL_API_ENDPOINT"],
                    api_key=os.environ["MISTRAL_API_KEY"],
                    temperature=0, max_tokens=COMPLETION_TOKENS,
                    streaming=True)

llm_cohere = ChatCohere(
    base_url=os.environ["COHERE_API_ENDPOINT"],
    cohere_api_key=os.environ["COHERE_API_KEY"],
    streaming=True,
    max_tokens=COMPLETION_TOKENS,
    temperature=0
)

Let's call the invoke method and see if they work

In [32]:
llm_mistral.invoke("who are you?").content

'I am a language model trained by the Mistral AI team. I am designed to understand and generate human-like text based on the input I receive. I can assist with a wide range of tasks, from answering questions and providing information to helping with brainstorming and creativity.'

In [31]:
llm_cohere.invoke("who are you?").content

NameError: name 'llm_cohere' is not defined

Great, now let's add the same RAG chain that we used before with the Azure OpenAI models

In [38]:
mistral_chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the input variables above to the prompt template
    | llm_mistral   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

cohere_chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the input variables above to the prompt template
    | llm_cohere   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

Since they also both support streaming, we can use the `stream` method from langchain as well

In [39]:
for chunk in mistral_chain.stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

Several documents suggest different medications that can reduce inflammation in the lungs.

1. Interleukin-22 (IL-22) is a cytokine that can protect the lung during infection by reducing pulmonary inflammation and promoting tight junction formation. Administering recombinant IL-22 in vivo has been shown to reduce inflammation and fluid leak into the lung [[1]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6917921/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-07-09T13:52:04Z&st=2024-07-09T05:52:04Z&spr=https&sig=9Nx31tWOzf6CWylUZnGaciT9VDWVJSJ9vQulMcshm7Q%3D).

2. Chloroquine, at clinically achievable concentrations in the lung (100 μM), has been found to inhibit the lipopolysaccharide-induced release of pro-inflammatory cytokines such as TNF-α, IL-6, CCL2, and CCL3 in human lung parenchymal explants. This suggests that chloroquine may mitigate the cytokine storm associated with severe pneumonia caused by coronaviruses [[2]](https://doi.org/10.1093/cid/ciaa546;%20https://www.ncbi.nlm.nih.gov

In [42]:
for chunk in cohere_chain.stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

- Chloroquine
- Escin
- Interleukin-22 (IL-22)
- Low doses of radiation therapy (RT)
- Tocilizumab
- Tanshinone IIA (TIIA)
- Dexamethasone
- Natural herbal medicine (NHM)
- Corticosteroids

The two models, Mistral Large and Command R+, as demonstrated, are capable of capturing context more effectively than GPT-3.5, resulting in superior responses. In terms of speed and breath of response, they are comparable to GPT-4.

As we will see in the next notebooks, we will need a bit more than just understanding all the context in order to build a multi-agentic bot architecture (parallel function calling for example), but for now it is good to understand the options available in terms of: 
- Context understanding
- Speed of response
- Quality of response
- Price

Check out these Model Benchmark charts provided by Azure AI Studio: https://ai.azure.com/explore/benchmarks

# Summary
##### By using OpenAI, the answers to user questions are way better than taking just the results from Azure AI Search. So the summary is:
- Utilizing Azure AI Search, we conduct a multi-index hybrid search that identifies the top chunks of documents from each index.
- Subsequently, Azure OpenAI utilizes these extracted chunks as context, comprehends the content, and employs it to deliver optimal answers.
- Best of two worlds!

##### Important observations on this notebook:

1) Answers with GPT-3.5 are less quality but fast
2) Answers with GPT-3.5 sometimes failed on provinding citations in the right format
3) Answers with GPT-4 are great quality but slower
4) Answers with GPT-4 always provide good and diverse citations in the right format
5) Models like Mistral Large or Cohere Command R+ provide, so far, a similar experience to GPT-4
5) Streaming the answers improves the user experience big time!

# NEXT
In the next notebook, we are going to see how we can treat complex and large documents separately, also using Vector Search